# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 24 2022 8:30AM,249195,10,ctf0011465,"Citieffe, Inc.",Released
1,Oct 24 2022 8:30AM,249196,10,0086098242,ISDIN Corporation,Released
2,Oct 24 2022 8:30AM,249196,10,0086098244,ISDIN Corporation,Released
3,Oct 24 2022 8:30AM,249196,10,0086098243,ISDIN Corporation,Released
4,Oct 24 2022 8:30AM,249196,10,0086098245,ISDIN Corporation,Released
5,Oct 24 2022 8:30AM,249196,10,0086098246,ISDIN Corporation,Released
6,Oct 24 2022 8:30AM,249196,10,0086098241,ISDIN Corporation,Released
7,Oct 24 2022 8:30AM,249196,10,0086098240,ISDIN Corporation,Released
8,Oct 24 2022 8:30AM,249196,10,0086098257,ISDIN Corporation,Released
9,Oct 24 2022 8:30AM,249196,10,0086098258,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
11,249199,Released,1
12,249200,Released,1
13,249201,Released,1
14,249202,Released,1
15,249203,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Released
id,,
249199,NaN,1.0
249200,NaN,1.0
249201,NaN,1.0
249202,NaN,1.0
249203,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Released
id,,
249157,0.0,3.0
249184,8.0,2.0
249190,0.0,1.0
249191,0.0,1.0
249192,0.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Executing does not exist yet


ShipmentStatus,Completed,Released
id,,
249157,0,3
249184,8,2
249190,0,1
249191,0,1
249192,0,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Released
0,249157,0,3
1,249184,8,2
2,249190,0,1
3,249191,0,1
4,249192,0,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Executing does not exist yet


ShipmentStatus,id,Completed,Released
0,249157,,3
1,249184,8,2
2,249190,,1
3,249191,,1
4,249192,,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 24 2022 8:30AM,249195,10,"Citieffe, Inc."
1,Oct 24 2022 8:30AM,249196,10,ISDIN Corporation
13,Oct 22 2022 11:01AM,249203,22,"NBTY Global, Inc."
14,Oct 22 2022 11:00AM,249202,22,"NBTY Global, Inc."
15,Oct 22 2022 11:00AM,249201,22,"NBTY Global, Inc."
16,Oct 21 2022 5:34PM,249200,21,"NBTY Global, Inc."
17,Oct 21 2022 5:07PM,249199,21,"NBTY Global, Inc."
18,Oct 21 2022 5:05PM,249198,21,"NBTY Global, Inc."
19,Oct 21 2022 3:39PM,249194,20,Reliable 1 Laboratories LLC
20,Oct 21 2022 3:11PM,249193,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Released
0,Oct 24 2022 8:30AM,249195,10,"Citieffe, Inc.",,1
1,Oct 24 2022 8:30AM,249196,10,ISDIN Corporation,,12
2,Oct 22 2022 11:01AM,249203,22,"NBTY Global, Inc.",,1
3,Oct 22 2022 11:00AM,249202,22,"NBTY Global, Inc.",,1
4,Oct 22 2022 11:00AM,249201,22,"NBTY Global, Inc.",,1
5,Oct 21 2022 5:34PM,249200,21,"NBTY Global, Inc.",,1
6,Oct 21 2022 5:07PM,249199,21,"NBTY Global, Inc.",,1
7,Oct 21 2022 5:05PM,249198,21,"NBTY Global, Inc.",,1
8,Oct 21 2022 3:39PM,249194,20,Reliable 1 Laboratories LLC,,1
9,Oct 21 2022 3:11PM,249193,21,"NBTY Global, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Completed
0,Oct 24 2022 8:30AM,249195,10,"Citieffe, Inc.",1,
1,Oct 24 2022 8:30AM,249196,10,ISDIN Corporation,12,
2,Oct 22 2022 11:01AM,249203,22,"NBTY Global, Inc.",1,
3,Oct 22 2022 11:00AM,249202,22,"NBTY Global, Inc.",1,
4,Oct 22 2022 11:00AM,249201,22,"NBTY Global, Inc.",1,
5,Oct 21 2022 5:34PM,249200,21,"NBTY Global, Inc.",1,
6,Oct 21 2022 5:07PM,249199,21,"NBTY Global, Inc.",1,
7,Oct 21 2022 5:05PM,249198,21,"NBTY Global, Inc.",1,
8,Oct 21 2022 3:39PM,249194,20,Reliable 1 Laboratories LLC,1,
9,Oct 21 2022 3:11PM,249193,21,"NBTY Global, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Completed
0,Oct 24 2022 8:30AM,249195,10,"Citieffe, Inc.",1,
1,Oct 24 2022 8:30AM,249196,10,ISDIN Corporation,12,
2,Oct 22 2022 11:01AM,249203,22,"NBTY Global, Inc.",1,
3,Oct 22 2022 11:00AM,249202,22,"NBTY Global, Inc.",1,
4,Oct 22 2022 11:00AM,249201,22,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Executing does not exist yet


,Date,id,Warehouse,Customer,Released,Completed
0,Oct 24 2022 8:30AM,249195,10,"Citieffe, Inc.",1,NaN
1,Oct 24 2022 8:30AM,249196,10,ISDIN Corporation,12,NaN
2,Oct 22 2022 11:01AM,249203,22,"NBTY Global, Inc.",1,NaN
3,Oct 22 2022 11:00AM,249202,22,"NBTY Global, Inc.",1,NaN
4,Oct 22 2022 11:00AM,249201,22,"NBTY Global, Inc.",1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Completed
0,Oct 24 2022 8:30AM,249195,10,"Citieffe, Inc.",1,0.0
1,Oct 24 2022 8:30AM,249196,10,ISDIN Corporation,12,0.0
2,Oct 22 2022 11:01AM,249203,22,"NBTY Global, Inc.",1,0.0
3,Oct 22 2022 11:00AM,249202,22,"NBTY Global, Inc.",1,0.0
4,Oct 22 2022 11:00AM,249201,22,"NBTY Global, Inc.",1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Completed
Warehouse,,,
10,747575,15,8.0
19,498349,6,0.0
20,249194,1,0.0
21,1495171,6,0.0
22,747606,3,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Completed
0,10,747575,15,8.0
1,19,498349,6,0.0
2,20,249194,1,0.0
3,21,1495171,6,0.0
4,22,747606,3,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Completed
0,10,15,8.0
1,19,6,0.0
2,20,1,0.0
3,21,6,0.0
4,22,3,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,15.0
1,19,Released,6.0
2,20,Released,1.0
3,21,Released,6.0
4,22,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19,20,21,22
Status,,,,,
Completed,8.0,0.0,0.0,0.0,0.0
Released,15.0,6.0,1.0,6.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19,20,21,22
0,Completed,8.0,0.0,0.0,0.0,0.0
1,Released,15.0,6.0,1.0,6.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19,20,21,22
0,Completed,8.0,0.0,0.0,0.0,0.0
1,Released,15.0,6.0,1.0,6.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()